In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.layers import ConvLSTM1D, Flatten, Dense

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

class WeightLogger(tf.keras.callbacks.Callback):
    def __init__(self, layer_index=0):
        self.layer_index = layer_index
        self.weights_per_epoch = []

    def on_epoch_end(self, epoch, logs=None):
        weights = self.model.layers[self.layer_index].get_weights()[0]
        self.weights_per_epoch.append(weights.copy())


def load_and_prepare_data(csv_path, production_column='production', window_size=24):
    df = pd.read_csv(csv_path, sep=',')
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df.values)
    target_scaler = MinMaxScaler()
    target_scaler.fit(df[[production_column]])
    target_col_idx = df.columns.get_loc(production_column)
    x, y = [], []
    for i in range(window_size, len(data_scaled)):
        x.append(data_scaled[i-window_size:i])
        y.append(data_scaled[i, target_col_idx])
    x, y = np.array(x), np.array(y)
    train_split_index = int(0.8 * len(x))
    test_split_index = int(0.9 * len(x))
    x_train, y_train = x[:train_split_index], y[:train_split_index]
    x_test, y_test = x[train_split_index:test_split_index], y[train_split_index:test_split_index]
    x_val, y_val = x[test_split_index:], y[test_split_index:]
    x_train_conv = np.expand_dims(x_train, axis=2)
    x_test_conv = np.expand_dims(x_test, axis=2)
    x_val_conv = np.expand_dims(x_val, axis=2)
    return x_train_conv, y_train, x_test_conv, y_test, x_val_conv, y_val, df, target_scaler


def build_convlstm_model(lr, filters1, filters2, dense_units, input_shape):
    model = Sequential([
        ConvLSTM1D(filters=int(filters1), kernel_size=(1,), activation='tanh',
                   return_sequences=True, input_shape=input_shape),
        ConvLSTM1D(filters=int(filters2), kernel_size=(1,), activation='tanh', return_sequences=False),
        Flatten(),
        Dense(units=int(dense_units), activation='relu'),
        Dense(1, activation="linear")
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(loss="mae", optimizer=optimizer)
    return model


def train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                             epochs=50, batch_size=512, verbose=0, dataset_name="dataset"):
    stop_early = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    weight_logger = WeightLogger(layer_index=0)
    start_time = time.time()
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=verbose,
                        callbacks=[stop_early, weight_logger])
    training_time = time.time() - start_time
    w00 = [w[0, 0] for w in weight_logger.weights_per_epoch]
    plt.figure(figsize=(8, 4))
    plt.plot(w00)
    plt.xlabel("Époque")
    plt.ylabel("Poids [0,0]")
    plt.title("Évolution du poids [0,0]")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_poids_w00.png")
    plt.close()
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title("Loss par époque")
    plt.xlabel("Époque")
    plt.ylabel("MAE")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{dataset_name}_loss_curve.png")
    plt.close()
    return history, training_time, weight_logger


def inference_and_plot(model, x_test, y_test, target_scaler, dataset_name): 
    preds = model.predict(x_test)
    y_test_real = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_real = target_scaler.inverse_transform(preds.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(y_test_real, y_pred_real)
    mse = mean_squared_error(y_test_real, y_pred_real)
    r2 = r2_score(y_test_real, y_pred_real)
    plt.figure(figsize=(10, 6))
    plt.plot(y_test_real, label="Vrai")
    plt.plot(y_pred_real, label="Prévu")
    plt.legend()
    plt.title(f"{dataset_name} - MAE: {mae:.4f} | R²: {r2:.4f} | MSE: {mse:.4f}")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_courbe_perf.png")
    plt.close()
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test_real, y_pred_real, alpha=0.7, color='orange')
    plt.plot([min(y_test_real), max(y_test_real)], [min(y_test_real), max(y_test_real)], 'r--')
    plt.xlabel("Réel")
    plt.ylabel("Prédit")
    plt.title("Scatter plot")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_scatter_perf.png")
    plt.close()
    errors = y_test_real - y_pred_real
    plt.figure(figsize=(8, 4))
    plt.hist(errors, bins=30, color='orange', edgecolor='black')
    plt.title("Histogramme des erreurs")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_hist_errors.png")
    plt.close()
    # Corriger division par zéro pour les erreurs en pourcentage
    safe_y_test_real = np.where(y_test_real == 0, np.nan, y_test_real)
    
    df_stats = pd.DataFrame({
        "Y_test": y_test_real,
        "Y_pred": y_pred_real,
        "Error": errors,
        "Error_Percent": np.abs(errors) / safe_y_test_real * 100
    })
    
    # Sauvegarder les statistiques sans générer d'avertissements
    df_stats.describe().to_csv(f"{dataset_name}_stats_erreurs.csv")
    return mae, mse, r2


def objective(trial, input_shape, x_train, y_train, x_val, y_val):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    f1 = trial.suggest_int('filters1', 32, 128)
    f2 = trial.suggest_int('filters2', 32, 128)
    dense = trial.suggest_int('dense_units', 32, 128)
    model = build_convlstm_model(lr, f1, f2, dense, input_shape)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                        epochs=10, batch_size=256, verbose=0)
    return min(history.history['val_loss'])


def run_experiment(csv_path, dataset_name):
    x_train, y_train, x_test, y_test, x_val, y_val, df, target_scaler = load_and_prepare_data(csv_path)
    input_shape = x_train.shape[1:]
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, input_shape, x_train, y_train, x_val, y_val), n_trials=100)
    best = study.best_params
    model = build_convlstm_model(best['lr'], best['filters1'], best['filters2'], best['dense_units'], input_shape)
    history, training_time, _ = train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                                                         epochs=700, dataset_name=dataset_name)
    mae, mse, r2 = inference_and_plot(model, x_test, y_test, target_scaler, dataset_name)
    print(f"{dataset_name} — MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")


if __name__ == "__main__":
    run_experiment("scaled_dataset.csv", "scaled_dataset")


2025-06-16 14:31:57.213449: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-16 14:31:57.278001: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-16 14:31:59.061303: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/sismail/py_envs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning:

[I 2025-06-16 15:07:34,763] Trial 23 finished with value: 0.022561047226190567 and parameters: {'lr': 0.0025666586788811382, 'filters1': 117, 'filters2': 94, 'dense_units': 87}. Best is trial 17 with value: 0.022216087207198143.
[I 2025-06-16 15:09:25,312] Trial 24 finished with value: 0.025029629468917847 and parameters: {'lr': 0.004575221539881146, 'filters1': 101, 'filters2': 80, 'dense_units': 117}. Best is trial 17 with value: 0.022216087207198143.
[I 2025-06-16 15:11:32,683] Trial 25 finished with value: 0.022751614451408386 and parameters: {'lr': 0.0022161505993664996, 'filters1': 110, 'filters2': 68, 'dense_units': 97}. Best is trial 17 with value: 0.022216087207198143.
[I 2025-06-16 15:14:28,044] Trial 26 finished with value: 0.023471910506486893 and parameters: {'lr': 0.0012050151770691708, 'filters1': 121, 'filters2': 101, 'dense_units': 111}. Best is trial 17 with value: 0.022216087207198143.
[I 2025-06-16 15:16:10,695] Trial 27 finished with value: 0.022612079977989197 and

[I 2025-06-16 15:47:47,673] Trial 59 finished with value: 0.023892320692539215 and parameters: {'lr': 0.004512267924190785, 'filters1': 128, 'filters2': 95, 'dense_units': 43}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 15:48:21,371] Trial 60 finished with value: 0.02360609918832779 and parameters: {'lr': 0.003844787861276145, 'filters1': 125, 'filters2': 114, 'dense_units': 90}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 15:48:55,522] Trial 61 finished with value: 0.023183850571513176 and parameters: {'lr': 0.002855224564927874, 'filters1': 119, 'filters2': 99, 'dense_units': 67}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 15:49:26,492] Trial 62 finished with value: 0.022977206856012344 and parameters: {'lr': 0.001644652409743762, 'filters1': 122, 'filters2': 86, 'dense_units': 71}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 15:49:55,805] Trial 63 finished with value: 0.023679964244365692 and parameter

[I 2025-06-16 16:07:01,750] Trial 95 finished with value: 0.023327406495809555 and parameters: {'lr': 0.00017979329989510688, 'filters1': 123, 'filters2': 98, 'dense_units': 46}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 16:07:30,677] Trial 96 finished with value: 0.02423619106411934 and parameters: {'lr': 0.00015894244359846498, 'filters1': 101, 'filters2': 75, 'dense_units': 78}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 16:08:00,328] Trial 97 finished with value: 0.024277938529849052 and parameters: {'lr': 0.003373973826260171, 'filters1': 105, 'filters2': 90, 'dense_units': 90}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 16:08:30,163] Trial 98 finished with value: 0.02341333031654358 and parameters: {'lr': 0.0042437995006620885, 'filters1': 115, 'filters2': 72, 'dense_units': 68}. Best is trial 58 with value: 0.02218157798051834.
[I 2025-06-16 16:09:04,543] Trial 99 finished with value: 0.0241178497672081 and paramete

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
scaled_dataset — MAE: 1.9262, MSE: 12.1423, R²: 0.9624


In [2]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import optuna
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.layers import ConvLSTM1D, Flatten, Dense

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


class WeightLogger(tf.keras.callbacks.Callback):
    def __init__(self, layer_index=0):
        self.layer_index = layer_index
        self.weights_per_epoch = []

    def on_epoch_end(self, epoch, logs=None):
        weights = self.model.layers[self.layer_index].get_weights()[0]
        self.weights_per_epoch.append(weights.copy())


def load_and_prepare_data(csv_path, production_column='production', window_size=24):
    # === 1. Chargement des données ===
    df = pd.read_csv("full_dataset.csv",sep=";")  # Remplace par ton CSV réel
    # 1.1 Conversion de la colonne date
    df['date1'] = pd.to_datetime(df['date'], dayfirst=True)
    
    # 1.2 Création des features temporelles cycliques
    df['dayofweek'] = df['date1'].dt.dayofweek
    df['month'] = df['date1'].dt.month
    
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    df.drop(columns=['month', 'dayofweek'], inplace=True)
    
    # 1.3 Supprimer les colonnes non numériques avant normalisation
    df = df.select_dtypes(include=[np.number])
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df.values)
    target_scaler = MinMaxScaler()
    target_scaler.fit(df[[production_column]])
    target_col_idx = df.columns.get_loc(production_column)
    x, y = [], []
    for i in range(window_size, len(data_scaled)):
        x.append(data_scaled[i-window_size:i])
        y.append(data_scaled[i, target_col_idx])
    x, y = np.array(x), np.array(y)
    train_split_index = int(0.8 * len(x))
    test_split_index = int(0.9 * len(x))
    x_train, y_train = x[:train_split_index], y[:train_split_index]
    x_test, y_test = x[train_split_index:test_split_index], y[train_split_index:test_split_index]
    x_val, y_val = x[test_split_index:], y[test_split_index:]
    x_train_conv = np.expand_dims(x_train, axis=2)
    x_test_conv = np.expand_dims(x_test, axis=2)
    x_val_conv = np.expand_dims(x_val, axis=2)
    return x_train_conv, y_train, x_test_conv, y_test, x_val_conv, y_val, df, target_scaler


def build_convlstm_model(lr, filters1, filters2, dense_units, input_shape):
    model = Sequential([
        ConvLSTM1D(filters=int(filters1), kernel_size=(1,), activation='tanh',
                   return_sequences=True, input_shape=input_shape),
        ConvLSTM1D(filters=int(filters2), kernel_size=(1,), activation='tanh', return_sequences=False),
        Flatten(),
        Dense(units=int(dense_units), activation='relu'),
        Dense(1, activation="linear")
    ])
    optimizer = Adam(learning_rate=lr)
    model.compile(loss="mae", optimizer=optimizer)
    return model


def train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                             epochs=50, batch_size=512, verbose=0, dataset_name="dataset"):
    stop_early = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    weight_logger = WeightLogger(layer_index=0)
    start_time = time.time()
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=verbose,
                        callbacks=[stop_early, weight_logger])
    training_time = time.time() - start_time
    w00 = [w[0, 0] for w in weight_logger.weights_per_epoch]
    plt.figure(figsize=(8, 4))
    plt.plot(w00)
    plt.xlabel("Époque")
    plt.ylabel("Poids [0,0]")
    plt.title("Évolution du poids [0,0]")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_poids_w00.png")
    plt.close()
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title("Loss par époque")
    plt.xlabel("Époque")
    plt.ylabel("MAE")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{dataset_name}_loss_curve.png")
    plt.close()
    return history, training_time, weight_logger


def inference_and_plot(model, x_test, y_test, target_scaler, dataset_name):
    preds = model.predict(x_test)
    y_test_real = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred_real = target_scaler.inverse_transform(preds.reshape(-1, 1)).flatten()
    mae = mean_absolute_error(y_test_real, y_pred_real)
    mse = mean_squared_error(y_test_real, y_pred_real)
    r2 = r2_score(y_test_real, y_pred_real)
    plt.figure(figsize=(10, 6))
    plt.plot(y_test_real, label="Vrai")
    plt.plot(y_pred_real, label="Prévu")
    plt.legend()
    plt.title(f"{dataset_name} - MAE: {mae:.4f} | R²: {r2:.4f} | MSE: {mse:.4f}")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_courbe_perf.png")
    plt.close()
    plt.figure(figsize=(6, 6))
    plt.scatter(y_test_real, y_pred_real, alpha=0.7, color='orange')
    plt.plot([min(y_test_real), max(y_test_real)], [min(y_test_real), max(y_test_real)], 'r--')
    plt.xlabel("Réel")
    plt.ylabel("Prédit")
    plt.title("Scatter plot")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_scatter_perf.png")
    plt.close()
    errors = y_test_real - y_pred_real
    plt.figure(figsize=(8, 4))
    plt.hist(errors, bins=30, color='orange', edgecolor='black')
    plt.title("Histogramme des erreurs")
    plt.grid(True)
    plt.savefig(f"{dataset_name}_hist_errors.png")
    plt.close()
    # Éviter la division par zéro pour les erreurs en pourcentage
    safe_y_test_real = np.where(y_test_real == 0, np.nan, y_test_real)
    
    df_stats = pd.DataFrame({
        "Y_test": y_test_real,
        "Y_pred": y_pred_real,
        "Error": errors,
        "Error_Percent": np.abs(errors) / safe_y_test_real * 100
    })
    
    # Sauvegarde sans warning
    df_stats.describe().to_csv(f"{dataset_name}_stats_erreurs.csv")
    return mae, mse, r2


def objective(trial, input_shape, x_train, y_train, x_val, y_val):
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    f1 = trial.suggest_int('filters1', 32, 128)
    f2 = trial.suggest_int('filters2', 32, 128)
    dense = trial.suggest_int('dense_units', 32, 128)
    model = build_convlstm_model(lr, f1, f2, dense, input_shape)
    history = model.fit(x_train, y_train, validation_data=(x_val, y_val),
                        epochs=10, batch_size=256, verbose=0)
    return min(history.history['val_loss'])


def run_experiment(csv_path, dataset_name):
    x_train, y_train, x_test, y_test, x_val, y_val, df, target_scaler = load_and_prepare_data(csv_path)
    input_shape = x_train.shape[1:]
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, input_shape, x_train, y_train, x_val, y_val), n_trials=100)
    best = study.best_params
    model = build_convlstm_model(best['lr'], best['filters1'], best['filters2'], best['dense_units'], input_shape)
    history, training_time, _ = train_and_evaluate_model(model, x_train, y_train, x_val, y_val,
                                                         epochs=700, dataset_name=dataset_name)
    mae, mse, r2 = inference_and_plot(model, x_test, y_test, target_scaler, dataset_name)
    print(f"{dataset_name} — MAE: {mae:.4f}, MSE: {mse:.4f}, R²: {r2:.4f}")


if __name__ == "__main__":
    run_experiment("full_dataset.csv", "full_dataset")


[I 2025-06-16 16:22:24,469] A new study created in memory with name: no-name-95102ec0-8739-4090-8520-8f7a4641c36f
/home/sismail/py_envs/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[I 2025-06-16 16:22:56,344] Trial 0 finished with value: 0.02542615309357643 and parameters: {'lr': 0.0016303194000501296, 'filters1': 62, 'filters2': 88, 'dense_units': 110}. Best is trial 0 with value: 0.02542615309357643.
[I 2025-06-16 16:23:28,067] Trial 1 finished with value: 0.02477753907442093 and parameters: {'lr': 0.0023154647103033697, 'filters1': 88, 'filters2': 88, 'dense_units': 49}. Best is trial 1 with value: 0.02477753907442093.
[I 2025-06-16 16:24:01,058] Trial 2 finished with value: 0.026397064328193665 and parameters: {'lr': 0.0001109385947742981, 'filters1': 123,

[I 2025-06-16 16:40:55,378] Trial 35 finished with value: 0.025310836732387543 and parameters: {'lr': 0.002807401638013155, 'filters1': 128, 'filters2': 63, 'dense_units': 99}. Best is trial 33 with value: 0.02439337596297264.
[I 2025-06-16 16:41:26,174] Trial 36 finished with value: 0.024792322888970375 and parameters: {'lr': 0.007216307833200836, 'filters1': 115, 'filters2': 52, 'dense_units': 114}. Best is trial 33 with value: 0.02439337596297264.
[I 2025-06-16 16:41:51,329] Trial 37 finished with value: 0.02473204955458641 and parameters: {'lr': 0.0019283347289159303, 'filters1': 77, 'filters2': 38, 'dense_units': 100}. Best is trial 33 with value: 0.02439337596297264.
[I 2025-06-16 16:42:24,722] Trial 38 finished with value: 0.02535884827375412 and parameters: {'lr': 0.0012024112796546122, 'filters1': 123, 'filters2': 68, 'dense_units': 80}. Best is trial 33 with value: 0.02439337596297264.
[I 2025-06-16 16:42:51,466] Trial 39 finished with value: 0.025547169148921967 and paramete

[I 2025-06-16 16:59:54,224] Trial 72 finished with value: 0.024544134736061096 and parameters: {'lr': 0.006659516559571663, 'filters1': 36, 'filters2': 74, 'dense_units': 56}. Best is trial 63 with value: 0.024100443348288536.
[I 2025-06-16 17:00:20,748] Trial 73 finished with value: 0.024779019877314568 and parameters: {'lr': 0.008109243474056602, 'filters1': 48, 'filters2': 79, 'dense_units': 41}. Best is trial 63 with value: 0.024100443348288536.
[I 2025-06-16 17:00:55,180] Trial 74 finished with value: 0.026989640668034554 and parameters: {'lr': 0.00019310804529491747, 'filters1': 112, 'filters2': 85, 'dense_units': 53}. Best is trial 63 with value: 0.024100443348288536.
[I 2025-06-16 17:01:32,252] Trial 75 finished with value: 0.024751538410782814 and parameters: {'lr': 0.0026159248517936182, 'filters1': 126, 'filters2': 89, 'dense_units': 65}. Best is trial 63 with value: 0.024100443348288536.
[I 2025-06-16 17:02:07,997] Trial 76 finished with value: 0.024725498631596565 and para

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
full_dataset — MAE: 3.5607, MSE: 36.2218, R²: 0.8889
